In [53]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd

from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

In [54]:
df = pd.read_csv('../data/all_MOFs_screening_data.csv')
df.head()

,MOFname,CO2_uptake_P0.15bar_T298K [mmol/g],CO2_uptake_error_P0.15bar_T298K [mmol/g],heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],heat_adsorption_error_CO2_P0.15bar_T298K [kcal/mol],excess_CO2_uptake_P0.15bar_T298K [mmol/g],CO2_uptake_P0.10bar_T363K [mmol/g],CO2_uptake_error_P0.10bar_T363K [mmol/g],heat_adsorption_CO2_P0.10bar_T363K [kcal/mol],heat_adsorption_error_CO2_P0.10bar_T363K [kcal/mol],...,void_fraction,void_volume [cm^3/g],largest_free_sphere_diameter [A],largest_included_sphere_along_free_sphere_path_diameter [A],largest_included_sphere_diameter [A],functional_groups,metal_linker,organic_linker1,organic_linker2,topology
0,str_m5_o16_o16_sra_sym.77,5.955197,0.117622,9.223227,0.546586,5.955127,0.840043,0.295803,7.972884,5.106942,...,0.14835,0.1480,4.61370,3.78047,4.61370,Pr,9,16,16,sra
1,str_m5_o16_o16_sra_sym.37,5.715251,0.034598,8.391884,0.149962,5.715251,0.901656,0.283054,8.343676,4.785696,...,0.16099,0.1624,4.10210,3.39756,4.10055,OMe,9,16,16,sra
2,str_m5_o13_o18_sra_sym.149,5.524486,0.155094,8.674098,0.753841,5.524486,0.688965,0.191963,7.546794,3.803472,...,0.17980,0.1810,4.24711,3.26179,4.24603,NO2,9,13,18,sra
3,str_m5_o2_o18_sra_sym.4,5.517258,0.084320,10.791553,0.514052,5.517258,1.881691,0.166854,9.246778,1.511943,...,0.10245,0.0922,4.11419,2.93595,3.63925,H,9,2,18,sra
4,str_m5_o16_o16_sra_sym.31,5.451593,0.051989,8.711414,0.260774,5.451476,0.797098,0.254966,8.247735,4.814885,...,0.15634,0.1583,4.37077,3.49681,4.16451,Et,9,16,16,sra


In [55]:
df = df.drop(columns=df.columns[2:28])
df = df.drop(columns=['MOFname', 'topology', 'functional_groups'])

In [56]:
# Sample 10% of the data
#df_sample = df.sample(frac=0.1, random_state=42)

y = df['CO2_uptake_P0.15bar_T298K [mmol/g]']
X_df = df.drop(columns=['CO2_uptake_P0.15bar_T298K [mmol/g]'])
X_df

,CO2/N2_selectivity,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],largest_free_sphere_diameter [A],largest_included_sphere_along_free_sphere_path_diameter [A],largest_included_sphere_diameter [A],metal_linker,organic_linker1,organic_linker2
0,109.786792,2473.186302,1493.011840,613.30,0.14835,0.1480,4.61370,3.78047,4.61370,9,16,16
1,126.719639,2419.885159,1444.796800,0.00,0.16099,0.1624,4.10210,3.39756,4.10055,9,16,16
2,141.713366,2514.627698,1504.683120,0.00,0.17980,0.1810,4.24711,3.26179,4.24603,9,13,18
3,274.702092,2128.612920,1424.810240,0.00,0.10245,0.0922,4.11419,2.93595,3.63925,9,2,18
4,101.853896,2415.251225,1436.905520,1347.04,0.15634,0.1583,4.37077,3.49681,4.16451,9,16,16
...,...,...,...,...,...,...,...,...,...,...,...,...
324421,0.000000,949.067112,753.787520,0.00,0.00000,0.0000,3.15041,1.39118,2.95451,2,10,18
324422,0.000000,1202.182553,1042.538240,0.00,0.00000,0.0000,3.20407,1.78334,3.17435,3,12,24
324423,24.131770,1188.302573,1001.700216,0.00,0.00000,0.0000,3.36153,1.71487,3.36153,3,4,24
324424,6.071818,1506.660363,1493.296496,0.00,0.01108,0.0000,3.72847,1.97519,3.11676,10,42,46


In [57]:

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply the scaler to the numerical columns
X = scaler.fit_transform(X_df)

In [58]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Convert DataFrames to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset
dataset = CustomDataset(X_tensor, y_tensor)

# Define the size of the validation set
val_size = int(0.2 * len(dataset))  # 20% for validation
train_size = len(dataset) - val_size

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader objects for training and validation sets
trainloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=10, shuffle=False)

In [52]:
input_size = X.shape[1]

'''
    Multilayer Perceptron
'''
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

if __name__ == '__main__':

    # Hyperparameters
    batch_size = 10
    learning_rate = 1e-4
    epochs = 5

    # set fixed random seed
    torch.manual_seed(42)

    # Initializing Net
    mlp = MLP()

    # defining loss function
    loss_function = nn.MSELoss()
    # defining optimizer
    optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

    with open('training_log.txt', 'w') as f:
            # training loop
        for epoch in range(epochs):

            print('Epoch %s' % (epoch+1), file=f)

            epoch_loss = 0.0
            num_batches = 0

            # iterate over the data
            for i, data in enumerate(trainloader, 0):

                # get data and ground truth
                inputs, targets = data

                # set gradients of all optimized tensors to zero
                optimizer.zero_grad()

                # forward pass of data through net
                outputs = mlp(inputs)

                # compute loss
                loss = loss_function(outputs, targets)

                # backward pass
                loss.backward()

                # optimizing parameters
                optimizer.step()
                epoch_loss += loss.item()
                num_batches += 1


            avg_epoch_loss = epoch_loss / num_batches
            print(f'Average Loss after epoch {epoch + 1}: {avg_epoch_loss:.3f}', file=f)

            mlp.eval()
            val_loss = 0.0
            with torch.no_grad():
                for data in valloader:
                    inputs, targets = data
                    outputs = mlp(inputs)
                    loss = loss_function(outputs, targets)
                    val_loss += loss.item()

            # Average validation loss over the entire validation set
            val_loss /= len(valloader)
            print(f'Validation Loss after epoch {epoch + 1}: {val_loss:.3f}', file=f)